In [1]:
# Importing required libraries
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import yfinance as yf
import pyfredapi as pf

import warnings
warnings.filterwarnings("ignore")

# Setting a path for different platforms  
if os.path.exists('G:/My Drive/REIF') | os.path.exists('G:/Мой диск/REIF'):
    directory = ''
else:
    from google.colab import drive
    drive.mount('/content/drive')
    directory = '/content/drive/MyDrive/REIF/'

# Fixing API key for the FRED
from configparser import ConfigParser
config = ConfigParser()
config.read(directory + 'config.cfg')
api_key = config.get('fred', 'api_key')

lags = [4, 13, 26]

### Target preparation

In [2]:
# https://haus.com/resources/the-common-haus-price-index  ->  Download full data series

# Importing original dataset
target = pd.read_csv(directory + 'Data/metro_full_series.csv')
target[['City', 'StatArea']] = target['msaname'].str.split(', ', n = 1, expand = True)
target['date'] = pd.to_datetime(target['date'])

# Creation of statarea dictionary
msas = target.drop_duplicates(['msa', 'msaname'])[['msa', 'msaname', 'City', 'StatArea']]
msas['MinDate'] = msas['msa'].apply(lambda x: target[target['msa'] == x]['date'].min())
msas['MaxDate'] = msas['msa'].apply(lambda x: target[target['msa'] == x]['date'].max())
msas['NumberOfEntries'] = msas['msa'].apply(lambda x: target[target['msa'] == x]['date'].count())
msas.to_parquet(directory + 'Data_prepared/statareas.parquet', index = False)

# Creating of target dataset, calculating of the WoW metrics and saving to parquet
target_cl = pd.DataFrame()
for msa in tqdm(msas['msa'].unique()):
    target_cl_msa = target[target['msa'] == msa][['date', 'msa', 'chpi', 'inventory']]
    for lag in lags:
        target_cl_msa[f'CHPI_{lag}_week_logdyn'] = np.log(target_cl_msa['chpi'] / target_cl_msa['chpi'].shift(lag))
        target_cl_msa[f'Inventory_{lag}_week_logdyn'] = np.log(target_cl_msa['inventory'] / target_cl_msa['inventory'].shift(lag))
    target_cl_msa['CHPI_6_month_fut'] = target_cl_msa['chpi'].shift(-26)
    target_cl = pd.concat([target_cl, target_cl_msa])
target_cl.dropna(inplace = True)
target_cl.rename(columns = {'chpi': 'CHPI', 'inventory': 'Inventory'}, inplace = True)

msa_anoms = []
for msa in tqdm(target_cl['msa'].unique()):
    if sum(target_cl[target_cl['msa'] == msa]['CHPI_4_week_logdyn'] > target_cl['CHPI_4_week_logdyn'].quantile(0.99)) +\
        sum(target_cl[target_cl['msa'] == msa]['CHPI_4_week_logdyn'] < target_cl['CHPI_4_week_logdyn'].quantile(0.01)) > 0:
        msa_anoms.append(msa)

target_fin = target_cl[~target_cl['msa'].isin(msa_anoms)]
target_fin.to_parquet(directory + 'Data_prepared/target.parquet', index = False)
target_fin

100%|██████████| 100/100 [00:00<00:00, 247.32it/s]


,date,msa,CHPI,Inventory,CHPI_4_week_logdyn,Inventory_4_week_logdyn,CHPI_13_week_logdyn,Inventory_13_week_logdyn,CHPI_26_week_logdyn,Inventory_26_week_logdyn,CHPI_6_month_fut
1345,2010-07-23,10740,176797.77,4708,-0.009829,0.014980,-0.024138,0.101639,0.002145,0.068113,156581.56
1346,2010-07-30,10740,176393.23,4738,-0.008935,0.011676,-0.028566,0.111052,0.012808,0.077882,156235.36
1347,2010-08-06,10740,175488.00,4728,-0.010134,0.016635,-0.061188,0.134015,0.007890,0.073718,155807.58
1348,2010-08-13,10740,175171.64,4709,-0.015085,0.030400,-0.063952,0.117969,-0.007034,0.223993,155399.05
1349,2010-08-20,10740,173994.03,4751,-0.015986,0.009092,-0.069643,0.105571,-0.013046,0.221776,154773.42
...,...,...,...,...,...,...,...,...,...,...,...
60160,2022-08-26,46140,214792.69,2496,-0.007887,0.052644,0.041396,0.269158,0.086160,0.482258,222930.27
60161,2022-09-02,46140,215249.77,2508,-0.003342,0.033242,0.038138,0.265599,0.090593,0.515355,225148.05
60162,2022-09-09,46140,216224.38,2540,0.002534,0.050465,0.038684,0.259730,0.092864,0.472211,224125.53
60163,2022-09-16,46140,216809.56,2519,0.007941,0.012785,0.036654,0.244814,0.093678,0.495332,224752.02


### Macro data preparation

In [60]:
# Define keys that can be imported through the API
ids_FRED = {'KeyRate': 'DFF', 'CPI': 'CPIAUCSL', 'VIX': 'VIXCLS', 'PPI': 'PCU44414441', 'MortgageRate30': 'MORTGAGE30US', 
            'Electricity': 'CUSR0000SEHF01', 'Water': 'CUSR0000SEHG', 'Plywood': 'WPU083', 'Steel': 'WPU101',
            'Glass': 'PCU3272132721', 'Concrete': 'PCU32733273', 'Unemployment': 'UNRATE', 'Yield10Y': 'DGS10',
            'Case-Shiller': 'SPCS20RSA'}
ids_YFINANCE = {'DJI': '^DJI', 'S&P500': '^GSPC'}
FRED_keys = [key for key in ids_FRED]
YFINANCE_keys = [key for key in ids_YFINANCE]

# Import data from FRED API
FRED = pd.DataFrame(columns = ['date'])
for key in FRED_keys:
    FRED_key = pf.get_series(series_id = ids_FRED[key], api_key = api_key)[['date', 'value']]
    FRED_key.rename(columns = {'value': key}, inplace = True)
    FRED = FRED.merge(FRED_key, on = 'date', how = 'outer')
FRED.sort_values('date', ascending = True, inplace = True)
FRED = FRED[FRED['date'] >= pd.to_datetime('2003-01-01')]

# Import data from Yahoo Finance API
YFINANCE = pd.DataFrame(columns = ['date'])
for key in YFINANCE_keys:
    YFINANCE_key = yf.download(tickers = ids_YFINANCE[key], period = "max", interval = "1d", group_by = 'ticker', auto_adjust = True, progress = False, threads = True)
    YFINANCE_key.reset_index(inplace = True)
    YFINANCE_key = YFINANCE_key[['Date', 'Close']]
    YFINANCE_key.rename(columns = {'Date': 'date', 'Close': key}, inplace = True)
    YFINANCE = YFINANCE.merge(YFINANCE_key, on = 'date', how = 'outer')
YFINANCE.sort_values('date', ascending = True, inplace = True)
YFINANCE = YFINANCE[YFINANCE['date'] >= pd.to_datetime('2003-01-01')]

# Merging all dataframes in one, interpolating nans and saving to parquet
macro = FRED.merge(YFINANCE, on = 'date', how = 'outer')
macro[macro.columns[1:]] = macro[macro.columns[1:]].interpolate(limit_direction = 'forward', limit_area = 'inside', axis = 0)
macro.dropna(inplace = True)
macro.sort_values('date', ascending = True, inplace = True)
macro.to_parquet(directory + 'Data_prepared/macro.parquet', index = False)
macro

,date,KeyRate,CPI,VIX,PPI,MortgageRate30,Electricity,Water,Plywood,Steel,Glass,Concrete,Unemployment,Yield10Y,Case-Shiller,DJI,S&P500
334,2003-12-01,1.03,185.500000,16.77,100.000000,5.945714,140.500000,120.000000,192.100000,128.400000,100.000000,100.000000,5.7,4.400000,150.758972,9899.049805,1070.119995
335,2003-12-02,0.97,185.525806,16.27,100.080645,5.964286,140.500000,120.019355,191.529032,128.580645,100.006452,100.029032,5.7,4.380000,150.819169,9853.639648,1066.619995
336,2003-12-03,0.98,185.551613,16.63,100.161290,5.982857,140.500000,120.038710,190.958065,128.761290,100.012903,100.058065,5.7,4.410000,150.879367,9873.419922,1064.729980
337,2003-12-04,0.99,185.577419,16.30,100.241935,6.001429,140.500000,120.058065,190.387097,128.941935,100.019355,100.087097,5.7,4.380000,150.939564,9930.820312,1069.719971
338,2003-12-05,0.98,185.603226,17.09,100.322581,6.020000,140.500000,120.077419,189.816129,129.122581,100.025806,100.116129,5.7,4.230000,150.999761,9862.679688,1061.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7574,2023-09-27,5.33,307.600600,18.22,209.213400,7.292857,268.468533,291.858667,285.430333,317.706600,166.734933,238.956800,3.8,4.610000,320.026643,33550.269531,4274.509766
7575,2023-09-28,5.33,307.605200,17.34,209.207300,7.310000,268.491400,291.884000,285.584500,317.472700,166.730700,238.966600,3.8,4.590000,320.094195,33666.339844,4299.700195
7576,2023-09-29,5.33,307.609800,17.52,209.201200,7.335714,268.514267,291.909333,285.738667,317.238800,166.726467,238.976400,3.8,4.590000,320.161748,33507.500000,4288.049805
7577,2023-09-30,5.33,307.614400,17.55,209.195100,7.361429,268.537133,291.934667,285.892833,317.004900,166.722233,238.986200,3.8,4.623333,320.229300,33482.783854,4288.163249


### Final datasets creation

In [2]:
# Importing both of the datasets 
target = pd.read_parquet(directory + 'Data_prepared/target.parquet')
macro = pd.read_parquet(directory + 'Data_prepared/macro.parquet')
target_2008 = macro[['date', 'Case-Shiller']]
target_2008['Case-Shiller'] *= 1000
# target_2008 = target_2008[target_2008['date'] <= pd.to_datetime('2012-01-01')]
macro.drop(columns = ['Case-Shiller'], inplace = True)
macro_dyn = pd.DataFrame(target['date'].unique(), columns = ['date']).merge(macro, on = 'date', how = 'inner')

# Filtering weekly dates for 2008 dataset
CS_dates = []
start_date = macro_dyn['date'].max()
for i in range(1, 1000):
    CS_dates.append(start_date - pd.DateOffset(weeks = i))
target_2008 = target_2008[target_2008['date'].isin(CS_dates)]
macro_dyn_2008 = target_2008.merge(macro, on = 'date', how = 'inner')

# Calculating log dynamics of the macro data with different lags
cols = macro_dyn.columns[1:].copy()
cols_2008 = macro_dyn_2008.columns[1:].copy()
for lag in lags:
    for col in cols:
        macro_dyn[f'{col}_{lag}_week_logdyn'] = np.log(macro_dyn[col] / macro_dyn[col].shift(lag))
    for col in cols_2008:
        macro_dyn_2008[f'{col}_{lag}_week_logdyn'] = np.log(macro_dyn_2008[col] / macro_dyn_2008[col].shift(lag))
macro_dyn_2008['Case-Shiller_6_month_fut'] = macro_dyn_2008['Case-Shiller'].shift(-26)
macro_dyn.dropna(inplace = True)
macro_dyn_2008.dropna(inplace = True)
macro_dyn.sort_values('date', ascending = True, inplace = True)
macro_dyn_2008.sort_values('date', ascending = True, inplace = True)
macro_dyn_2008.set_index('date', drop = True, inplace = True)

# Merging dataset with all columns
final_full = target.merge(macro_dyn, on = 'date', how = 'inner')
final_full.sort_values('date', ascending = True, inplace = True)
final_full.set_index('date', drop = True, inplace = True)
final_full.to_parquet(directory + 'Data_for_models/final_full.parquet')

# Dataset for EDA
eda_cols = [col for col in final_full.columns if (('_logdyn' not in col) & ('_fut' not in col)) & (col != 'msa')]
final_eda = final_full[eda_cols]
final_eda = final_eda[~final_eda.index.duplicated(keep = 'first')]
final_eda.drop(columns = ['CHPI', 'Inventory'], inplace = True)

# Importing aggregated index data for EDA
target_agg = pd.read_csv(directory + 'Data/metro_aggregates.csv')
target_agg['date'] = pd.to_datetime(target_agg['date'])

# Fixing the appearance of the aggregated index
fixes = {'\xa0': '', ' ': '', ',': ''}
for key in fixes:
    target_agg['CHPI'] = target_agg['CHPI'].str.replace(key, fixes[key])
target_agg['CHPI'] = target_agg['CHPI'].astype(int)
target_agg.set_index('date', drop = True, inplace = True)

# Joining and saving datasets for the EDA - we would loose 2 years of data but there is not other fast way to use aggregated index in the EDA
final_eda = target_agg.join(final_eda, how = 'inner')
final_eda.to_parquet(directory + 'Data_for_models/final_eda.parquet')

# Datasets for modelling (without and with inventory)
model_cols = [col for col in final_full.columns if col not in eda_cols] + ['CHPI']
final_model = final_full[model_cols]
final_model.set_index(['msa'], append = True, inplace = True)
q1 = final_model['CHPI_6_month_fut'].quantile(0.01)
q99 = final_model['CHPI_6_month_fut'].quantile(0.99)
final_model = final_model[final_model['CHPI_6_month_fut'].between(q1, q99)]
final_model[final_model.columns[~final_model.columns.str.startswith('Inventory')]].to_parquet(directory + 'Data_for_models/final_model.parquet')
final_model.to_parquet(directory + 'Data_for_models/final_model_with_inventory.parquet')

# Dataset for 2008 check of the model
drop_2008_cols = eda_cols + list(target.columns)
model_cols_2008 = [col for col in macro_dyn_2008.columns if col not in drop_2008_cols]
final_2008 = macro_dyn_2008[model_cols_2008]
final_2008.to_parquet(directory + 'Data_for_models/final_2008.parquet')
final_2008

,Case-Shiller,Case-Shiller_4_week_logdyn,KeyRate_4_week_logdyn,CPI_4_week_logdyn,VIX_4_week_logdyn,PPI_4_week_logdyn,MortgageRate30_4_week_logdyn,Electricity_4_week_logdyn,Water_4_week_logdyn,Plywood_4_week_logdyn,...,Water_26_week_logdyn,Plywood_26_week_logdyn,Steel_26_week_logdyn,Glass_26_week_logdyn,Concrete_26_week_logdyn,Unemployment_26_week_logdyn,Yield10Y_26_week_logdyn,DJI_26_week_logdyn,S&P500_26_week_logdyn,Case-Shiller_6_month_fut
date,,,,,,,,,,,,,,,,,,,,,
2004-06-04,164513.639505,0.013883,0.000000,0.003099,-0.077380,0.011486,0.021140,0.004973,0.003094,-0.101673,...,0.031015,0.045136,0.190887,-0.002962,0.034303,-0.019487,0.122239,0.037819,0.055875,175244.803005
2004-06-11,164999.370963,0.013282,-0.019803,0.002507,-0.205434,0.018759,-0.004974,0.004681,0.003492,-0.103564,...,0.031017,0.039986,0.197138,-0.002711,0.032500,-0.023670,0.125577,0.034173,0.053944,175800.283240
2004-06-18,165485.102420,0.012685,0.010050,0.001916,-0.209847,0.025948,0.001133,0.004390,0.003890,-0.105556,...,0.031019,0.034580,0.203231,-0.002461,0.030702,-0.027870,0.128700,0.013355,0.041703,176355.763475
2004-06-25,165970.833878,0.012093,0.028988,0.001325,-0.020203,0.033057,-0.011148,0.004099,0.004287,-0.107656,...,0.031021,0.028900,0.209172,-0.002212,0.028907,-0.032088,0.111099,0.004558,0.034564,176911.243710
2004-07-02,166441.510194,0.011650,0.233194,0.000969,-0.106818,0.035470,-0.016275,0.003950,0.004532,-0.101628,...,0.030964,0.023028,0.213432,-0.001998,0.027521,-0.035739,0.022574,-0.012277,0.015131,177466.723945
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-18,303830.583660,0.019777,0.000000,0.008597,-0.038874,0.086684,0.095639,0.009110,0.002642,0.068552,...,0.023158,0.299258,-0.012887,0.056339,0.052917,-0.277451,0.421213,-0.030086,-0.021114,307561.568443
2022-02-25,305304.044569,0.019562,0.000000,0.009495,-0.002534,0.092937,0.086676,0.014035,0.001776,0.068393,...,0.022337,0.369561,-0.016880,0.051509,0.053950,-0.272357,0.408006,-0.040200,-0.028048,306817.828752
2022-03-04,306560.718063,0.018692,0.000000,0.009312,0.320097,0.084114,0.055267,0.015908,0.001452,0.054107,...,0.021806,0.409519,-0.020030,0.044819,0.056298,-0.261103,0.268706,-0.050872,-0.046613,306150.007898
